#Install RDkit

In [ ]:
#!wget https://github.com/Jo-Hyung-Sik/GCN-HS/tree/master/utils.py -O utils.py
!mkdir results
!mkdir images

!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
# !time conda install -q -y -c conda-forge rdkit
!apt-get install -y python3-rdkit librdkit1 rdkit-data
!pip3 install rdkit

import sys
import os
import requests
import subprocess
import shutil
from logging import getLogger, StreamHandler, INFO


logger = getLogger(__name__)
logger.addHandler(StreamHandler())
logger.setLevel(INFO)

In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import SimilarityMaps, rdMolDraw2D
from rdkit.Chem import AllChem, rdDepictor, Draw, MolFromSmiles, QED, rdMolDescriptors, MolSurf

import numpy as np
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
from torchsummary import summary
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.utils.tensorboard import SummaryWriter

import os
import copy
import time
import gc
import sys

from numpy.polynomial.polynomial import polyfit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import argparse

sys.setrecursionlimit(50000)
torch.backends.cudnn.benchmark = True
# torch.set_default_tensor_type('torch.cuda.FloatTensor')
torch.nn.Module.dump_patches = True

# early stop

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            #self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            #print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter == self.patience:
                self.early_stop = True
            elif self.counter > self.patience:
                self.counter = 0
                self.early_stop = False
                self.best_score = None
        else:
            self.best_score = score
            #self.save_checkpoint(val_loss, model)
            self.counter = 0

'''
    def save_checkpoint(self, val_loss, model):
        Saves model when validation loss decrease.
        if self.verbose:
            #print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss
'''

"\n    def save_checkpoint(self, val_loss, model):\n        Saves model when validation loss decrease.\n        if self.verbose:\n            #print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')\n        torch.save(model.state_dict(), 'checkpoint.pt')\n        self.val_loss_min = val_loss\n"

#데이터 처리

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')

import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem, MolFromSmiles, Draw, rdDepictor, MolSurf
from rdkit.Chem.Draw import rdMolDraw2D, SimilarityMaps
import numpy as np
import os
import pickle
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import torch.utils.data as Data

import gc
import sys
import pickle
torch.manual_seed(950228) # for reproduce
sys.setrecursionlimit(50000)
torch.backends.cudnn.benchmark = True
# torch.set_default_tensor_type('torch.cuda.FloatTensor')
torch.set_default_tensor_type('torch.FloatTensor')
#from tensorboardX import SummaryWriter
torch.nn.Module.dump_patches = True
import copy
import pandas as pd
import argparse

from torch.utils.tensorboard import SummaryWriter
from numpy.polynomial.polynomial import polyfit
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
patience = 500

# result file path
result_file = "/content/drive/MyDrive/data/GAT-최적화-2layer-0.0002-0.0004.csv"

early_stopping = EarlyStopping(patience, verbose=True)

def get_splitted_dataset(ratios=[0.8, 0.2], seed=950228):
    train_data = pd.read_csv('/content/drive/MyDrive/data/colab용/처리끝/train-smi(opt)-knime.csv') # Open original dataset
    test_data = pd.read_csv('/content/drive/MyDrive/data/colab용/처리끝/test-smi(opt)-knime.csv') # Open original dataset

    #smiles = train_data['smiles']

    train, val = train_test_split(train_data, test_size=ratios[1], random_state=seed)

    test = test_data

    train, val, test = train.reset_index(drop=True), val.reset_index(drop=True), test.reset_index(drop=True)

    pd_save_column = pd.DataFrame(columns=['train_R2', 'train_MAE', 'train_MSE', 'val_R2', 'val_MAE', 'val_MSE', 'test_R2', 'test_MAE', 'test_MSE', 'n_layer', 'learning_rate', 'batch_size', 'out_dim', 'molvec_dim'])
    pd_save_column.to_csv(result_file)

    return train, val, test

datasets = get_splitted_dataset()

#Featurizer

In [ ]:
LIST_SYMBOLS = ['C', 'N', 'O', 'S', 'F', 'H', 'Si', 'P', 'Cl', 'Br',
            'Li', 'Na', 'K', 'Mg', 'Ca', 'Fe', 'As', 'Al', 'I', 'B',
            'V', 'Tl', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn',
            'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'Mn', 'Cr', 'Pt', 'Hg', 'Pb']

degrees = [0,1,2,3,4,5]

def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(
            x, allowable_set))
    return [int(x == s) for s in allowable_set]


def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return [int(x == s) for s in allowable_set]

def atom_feature(atom):
    return np.array(one_of_k_encoding_unk(atom.GetSymbol(), LIST_SYMBOLS) +
                    one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5, 6]) +
                    one_of_k_encoding(atom.GetTotalNumHs(), [0, 1, 2, 3, 4]) +
                    one_of_k_encoding(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5]) +
                    one_of_k_encoding(int(atom.GetIsAromatic()), [0, 1]))    # (40, 7, 5, 6, 2)

def bond_features(bond, use_chirality=True):
    bt = bond.GetBondType()
    bond_feats = [
        int(bt == Chem.rdchem.BondType.SINGLE), int(bt == Chem.rdchem.BondType.DOUBLE),
        int(bt == Chem.rdchem.BondType.TRIPLE), int(bt == Chem.rdchem.BondType.AROMATIC),
        int(bond.GetIsConjugated()),
        int(bond.IsInRing())
    ]
    if use_chirality:
        bond_feats = bond_feats + one_of_k_encoding_unk(
            str(bond.GetStereo()),
            ["STEREONONE", "STEREOANY", "STEREOZ", "STEREOE"])
    return np.array(bond_feats)

def mol2graph(smi):
    mol = Chem.MolFromSmiles(smi)
    atom_size = 150
    bond_size = 150
    atom_feature_array = np.zeros((atom_size, 60), dtype=np.uint8)
    bond_feature_array = np.empty((bond_size,10), dtype=np.uint8)
    atom_idx = {}
    mask = np.zeros((atom_size+1)) # +1 zero padding
    degree_list = {}
    atom_neighbor_list = []
    bond_neighbor_list = []

    for i, atom in enumerate(mol.GetAtoms()):
        feature = atom_feature(atom)
        atom_feature_array[i,:] = feature
        atom_idx[atom.GetIdx()] = atom
        mask[i] = 1.0
        degree_list[atom.GetIdx()] = atom.GetDegree()
        atom_neighbor_list.append(i)
    # zero padding
    atom_feature_array = np.append(atom_feature_array, np.zeros((1,60), dtype=np.uint8), axis=0)


    for i, bond in enumerate(mol.GetBonds()):
        atom1 = bond.GetBeginAtom().GetIdx()
        atom2 = bond.GetEndAtom().GetIdx()

        feature = bond_features(bond)
        bond_feature_array[i,:] = feature

        bond_neighbor_list.append([atom1, atom2])
    # zero padding
    bond_feature_array = np.append(bond_feature_array, np.zeros((1,10), dtype=np.uint8), axis=0)

    # neighbors
    atom_neighbors = np.zeros(((atom_size + 1), len(degrees)))
    bond_neighbors = np.zeros(((bond_size + 1), len(degrees)))

    atom_neighbors.fill(atom_size)
    bond_neighbors.fill(bond_size)

    all_bond_list = {}
    for i, atom_neighbor in enumerate(atom_neighbor_list):
        atom_neighbor_cnt = 0
        bond_list = []
        for j, bond_neighbor in enumerate(bond_neighbor_list):
            if atom_neighbor in bond_neighbor:
                remain_atom = bond_neighbor.copy()
                remain_atom.remove(atom_neighbor)
                atom_neighbors[i, atom_neighbor_cnt] = remain_atom[0]
                atom_neighbor_cnt += 1
                bond_list.append(j)

        # all_bond_list[atom_neighbor_cnt] = [bond_list]
        if atom_neighbor_cnt not in all_bond_list :
            # print(bond_list)
            all_bond_list[atom_neighbor_cnt] = [bond_list]
        else:
            all_bond_list[atom_neighbor_cnt].append(bond_list)

    all_bond_list = dict(sorted(all_bond_list.items()))
    bond_neighbor_cnt = 0
    for key, values in all_bond_list.items():
        for value in values:
            for i, val in enumerate(value):
                bond_neighbors[bond_neighbor_cnt, i] = val
            bond_neighbor_cnt += 1


    return {'mask': mask, 'atom_feature': atom_feature_array, 'bond_feature': bond_feature_array, 'atom_neighbor': atom_neighbors, 'bond_neighbor': bond_neighbors}

def get_smiles_array(smilesList):
    x_mask = []
    x_atom = []
    x_bonds = []
    x_atom_index = []
    x_bond_index = []
    for smiles in smilesList:
        feature_dicts = mol2graph(smiles)
        x_mask.append(feature_dicts['mask'])
        x_atom.append(feature_dicts['atom_feature'])
        x_bonds.append(feature_dicts['bond_feature'])
        x_atom_index.append(feature_dicts['atom_neighbor'])
        x_bond_index.append(feature_dicts['bond_neighbor'])
    return np.asarray(x_atom),np.asarray(x_bonds),np.asarray(x_atom_index),np.asarray(x_bond_index),np.asarray(x_mask)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class gatDataset(Dataset):
    def __init__(self, df):
        self.smiles = df['smiles']
        self.exp = df['exp'].values

        list_mask = list()
        list_atom_feature = list()
        list_bond_feature = list()
        list_atom_neighbor = list()
        list_bond_neighbor = list()
        for i, smiles in enumerate(self.smiles):
            result_dict = mol2graph(smiles)

            list_mask.append(result_dict['mask'])
            list_atom_feature.append(result_dict['atom_feature'])
            list_bond_feature.append(result_dict['bond_feature'])
            list_atom_neighbor.append(result_dict['atom_neighbor'])
            list_bond_neighbor.append(result_dict['bond_neighbor'])

        self.mask = np.asarray(list_mask)
        self.list_atom_feature = np.asarray(list_atom_feature)
        self.list_bond_feature = np.asarray(list_bond_feature)
        self.list_atom_neighbor = np.asarray(list_atom_neighbor)
        self.list_bond_neighbor = np.asarray(list_bond_neighbor)

    def __len__(self):
        return len(self.list_atom_feature)

    def __getitem__(self, index):
        return self.mask[index], self.list_atom_feature[index], self.list_bond_feature[index], self.list_atom_neighbor[index], self.list_bond_neighbor[index], self.exp[index]

sample_dataset = gatDataset(datasets[0])

ㅊ#Attentive layer

In [ ]:
class GAT(nn.Module):

    def __init__(self, args):
        super(GAT, self).__init__()

        self.fc = nn.Linear(args.in_dim, args.out_dim)
        self.neighbor_fc = nn.Linear(args.in_dim+args.in_bond_dim, args.out_dim)
        self.GRUCell = nn.GRUCell(args.out_dim, args.out_dim)
        self.align = nn.Linear(2*args.out_dim,1)
        self.attend = nn.Linear(args.out_dim, args.out_dim)

        self.mol_GRUCell = nn.GRUCell(args.out_dim, args.out_dim)
        self.mol_align = nn.Linear(2*args.out_dim,1)
        self.mol_attend = nn.Linear(args.out_dim, args.out_dim)

        self.dropout = nn.Dropout(p=args.dropout)
        self.output = nn.Linear(args.out_dim, args.output_unit)

        self.radius = args.radius
        self.T = args.T
        self.batch_size = args.batch_size

    def forward(self, atom_list, bond_list, atom_degree_list, bond_degree_list, atom_mask):
        atom_mask = atom_mask.unsqueeze(2)
        batch, mol_length, num_atom_feat = atom_list.size()
        atom_feature = F.leaky_relu(self.fc(atom_list))

        bond_neighbor = [bond_list[i][bond_degree_list[i]] for i in range(batch)]
        atom_neighbor = [atom_list[i][atom_degree_list[i]] for i in range(batch)]
        bond_neighbor = torch.stack(bond_neighbor, dim=0)
        atom_neighbor = torch.stack(atom_neighbor, dim=0)
        neighbor_feature = torch.cat([atom_neighbor, bond_neighbor],dim=-1)
        neighbor_feature = F.leaky_relu(self.neighbor_fc(neighbor_feature))

        # generate mask to eliminate the influence of blank atoms
        attend_mask = atom_degree_list.clone()
        attend_mask[attend_mask != mol_length-1] = 1
        attend_mask[attend_mask == mol_length-1] = 0
        # attend_mask = attend_mask.type(torch.cuda.FloatTensor).unsqueeze(-1)
        attend_mask = attend_mask.type(torch.FloatTensor).unsqueeze(-1)

        softmax_mask = atom_degree_list.clone()
        softmax_mask[softmax_mask != mol_length-1] = 0
        softmax_mask[softmax_mask == mol_length-1] = -9e8
        # softmax_mask = softmax_mask.type(torch.cuda.FloatTensor).unsqueeze(-1)
        softmax_mask = softmax_mask.type(torch.FloatTensor).unsqueeze(-1)

        batch, mol_length, max_neighbor_num, fingerprint_dim = neighbor_feature.shape
        atom_feature_expand = atom_feature.unsqueeze(-2).expand(batch, mol_length, max_neighbor_num, fingerprint_dim)
        feature_align = torch.cat([atom_feature_expand, neighbor_feature],dim=-1)
        align_score = F.leaky_relu(self.align(self.dropout(feature_align))) + softmax_mask

        attention_weight = F.softmax(align_score,-2) * attend_mask
        neighbor_feature_transform = self.attend(self.dropout(neighbor_feature))

        context = torch.sum(torch.mul(attention_weight,neighbor_feature_transform),-2)
        context = F.elu(context)
        context_reshape = context.view(batch*mol_length, fingerprint_dim)

        atom_feature_reshape = atom_feature.view(batch*mol_length, fingerprint_dim)
        atom_feature_reshape = self.GRUCell(context_reshape, atom_feature_reshape) # readout
        atom_feature = atom_feature_reshape.view(batch, mol_length, fingerprint_dim)

        #do nonlinearity
        activated_features = F.relu(atom_feature)

        # neigbhor atom attention in radius
        for d in range(self.radius-1):
            neighbor_feature = [activated_features[i][atom_degree_list[i]] for i in range(self.batch_size)]
            neighbor_feature = torch.stack(neighbor_feature, dim=0)
            atom_feature_expand = activated_features.unsqueeze(-2).expand(self.batch_size, mol_length, max_neighbor_num, fingerprint_dim)

            feature_align = torch.cat([atom_feature_expand, neighbor_feature],dim=-1)
            align_score = F.leaky_relu(self.align(self.dropout(feature_align))) + softmax_mask

            attention_weight = F.softmax(align_score,-2) * attend_mask
            neighbor_feature_transform = self.attend(self.dropout(neighbor_feature))

            context = torch.sum(torch.mul(attention_weight,neighbor_feature_transform),-2)
            context = F.elu(context)
            context_reshape = context.view(batch*mol_length, fingerprint_dim)

            atom_feature_reshape = self.GRUCell(context_reshape, atom_feature_reshape) # readout
            atom_feature = atom_feature_reshape.view(batch, mol_length, fingerprint_dim)

            activated_features = F.relu(atom_feature)

        mol_feature = torch.sum(activated_features * atom_mask, dim=-2)
        activated_features_mol = F.relu(mol_feature)

        mol_softmax_mask = atom_mask.clone()
        mol_softmax_mask[mol_softmax_mask == 0] = -9e8
        mol_softmax_mask[mol_softmax_mask == 1] = 0
        # mol_softmax_mask = mol_softmax_mask.type(torch.cuda.FloatTensor)
        mol_softmax_mask = mol_softmax_mask.type(torch.FloatTensor)

        for t in range(self.T):
            mol_prediction_expand = activated_features_mol.unsqueeze(-2).expand(batch, mol_length, fingerprint_dim)

            mol_align = torch.cat([mol_prediction_expand, activated_features], dim=-1)
            mol_align_score = F.leaky_relu(self.mol_align(mol_align)) + mol_softmax_mask

            mol_attention_weight = F.softmax(mol_align_score,-2) * atom_mask
            activated_features_transform = self.mol_attend(self.dropout(activated_features))

            mol_context = torch.sum(torch.mul(mol_attention_weight,activated_features_transform),-2)
            mol_context = F.elu(mol_context)
            mol_feature = self.mol_GRUCell(mol_context, mol_feature)

            # do nonlinearity
            activated_features_mol = F.relu(mol_feature)

        mol_prediction = self.output(self.dropout(mol_feature))

        return atom_feature, mol_prediction

# Train, Validation, Test

In [ ]:
def train(model, dataloader, optimizer, criterion, args, **kwargs):
    epoch_train_loss = 0
    list_train_loss = list()
    tr_list_y, tr_list_pred_y = list(), list()
    for batch_idx, batch in enumerate(dataloader):
        x_mask, x_atom, x_bond, x_atom_index, x_bond_index, y = batch[0].float(), batch[1].float(), batch[2].float(), batch[3].long(), batch[4].long(), batch[5].float()
        x_mask, x_atom, x_bond, x_atom_index, x_bond_index, y = x_mask.to(args.device), x_atom.to(args.device), x_bond.to(args.device), x_atom_index.to(args.device), x_bond_index.to(args.device), y.to(args.device)

        model.train()
        optimizer.zero_grad()

        atom_prediction, mol_prediction = model(x_atom, x_bond, x_atom_index, x_bond_index, x_mask)

        train_loss = criterion(mol_prediction, y)
        epoch_train_loss += train_loss.item()
        list_train_loss.append({'epoch':batch_idx/len(dataloader)+kwargs['epoch'], 'train_loss':train_loss.item()})
        train_loss.backward()
        optimizer.step()
        tr_list_y += y.cpu().detach().numpy().tolist()
        tr_list_pred_y += mol_prediction.cpu().detach().numpy().tolist()

    tr_r2 = r2_score(tr_list_y, tr_list_pred_y)
    tr_mae = mean_absolute_error(tr_list_y, tr_list_pred_y)
    tr_mse = mean_squared_error(tr_list_y, tr_list_pred_y)

    return model, list_train_loss, tr_r2, tr_mae, tr_mse, train_loss

def eval(model, dataloader, criterion, args):
    epoch_eval_loss = 0
    cnt_iter = 0
    eval_list_y, eval_list_pred_y = list(), list()
    for batch_idx, batch in enumerate(dataloader):
        x_mask, x_atom, x_bond, x_atom_index, x_bond_index, y = batch[0].float(), batch[1].float(), batch[2].float(), batch[3].long(), batch[4].long(), batch[5].float()
        x_mask, x_atom, x_bond, x_atom_index, x_bond_index, y = x_mask.to(args.device), x_atom.to(args.device), x_bond.to(args.device), x_atom_index.to(args.device), x_bond_index.to(args.device), y.to(args.device)

        model.eval()

        atom_prediction, mol_prediction = model(x_atom, x_bond, x_atom_index, x_bond_index, x_mask)

        eval_loss = criterion(mol_prediction, y)
        epoch_eval_loss += eval_loss.item()
        cnt_iter += 1
        eval_list_y += y.cpu().detach().numpy().tolist()
        eval_list_pred_y += mol_prediction.cpu().detach().numpy().tolist()

    eval_r2 = r2_score(eval_list_y, eval_list_pred_y)
    eval_mae = mean_absolute_error(eval_list_y, eval_list_pred_y)
    eval_mse = mean_squared_error(eval_list_y, eval_list_pred_y)

    return epoch_val_loss/cnt_iter , val_r2, val_mae, val_mse

def experiment(partition, args):
    ts = time.time()
    tf_writer = SummaryWriter()

    model = GAT(args)
    model.to(args.device)
    criterion = nn.MSELoss()

    trainable_parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = optim.Adam(trainable_parameters, lr=args.lr, weight_decay=args.l2_coef)
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

    # Train, Validate, Evaluate
    list_train_loss = list()
    list_val_loss = list()
    list_mae = list()
    list_std = list()
    list_mse = list()
    list_r2 = list()
    list_tr_r2 = list()
    list_tr_mae = list()
    list_tr_mse = list()
    list_val_r2 = list()
    list_val_mae = list()
    list_val_mse = list()

    # save parameter
    save_mae = list()
    save_mse = list()
    save_r2 = list()
    save_tr_r2 = list()
    save_tr_mae = list()
    save_tr_mse = list()
    save_val_r2 = list()
    save_val_mae = list()
    save_val_mse = list()
    save_result = list()
    save_epoch = list()

    args.best_val_loss = 10000
    for epoch in range(args.epoch):
        model, train_losses , tr_r2, tr_mae, tr_mse, tr_loss= train(model, partition['train'], optimizer, criterion, args, **{'epoch':epoch})
        val_loss, val_r2, val_mae, val_mse= eval(model, partition['val'], criterion, args)
        ts_r2, ts_mae, ts_mse, std, true_y, pred_y = eval(model, partition['test'], args, **{'epoch':epoch})

        list_train_loss += train_losses
        list_val_loss.append({'epoch':epoch, 'val_loss':val_loss})
        list_r2.append({'epoch':epoch, 'r2':ts_r2})
        list_mae.append({'epoch':epoch, 'mae':ts_mae})
        list_mse.append({'epoch':epoch, 'mse':ts_mse})
        list_std.append({'epoch':epoch, 'std':std})
        list_tr_r2.append({'epoch':epoch, 'r2':tr_r2})
        list_tr_mae.append({'epoch':epoch, 'mae':tr_mae})
        list_tr_mse.append({'epoch':epoch, 'mse':tr_mse})
        list_val_r2.append({'epoch':epoch, 'r2':val_r2})
        list_val_mae.append({'epoch':epoch, 'mae':val_mae})
        list_val_mse.append({'epoch':epoch, 'mse':val_mse})

        save_tr_r2.append(tr_r2)
        save_tr_mae.append(tr_mae)
        save_tr_mse.append(tr_mse)
        save_val_r2.append(val_r2)
        save_val_mae.append(val_mae)
        save_val_mse.append(val_mse)
        save_r2.append(ts_r2)
        save_mae.append(ts_mae)
        save_mse.append(ts_mse)
        save_epoch.append(epoch)

        tf_writer.add_scalar('Loss/train', tr_loss, epoch)
        tf_writer.add_scalar('Loss/val', val_loss, epoch)
        tf_writer.add_scalar('MSE/test', ts_mae, epoch)

        if args.best_val_loss > val_loss or epoch==0:
            args.best_val_loss
            args.best_epoch = epoch
            args.best_r2 = ts_r2
            args.best_mae = ts_mae
            args.best_mse = ts_mse
            args.best_std = std
            args.best_true_y = true_y
            args.best_pred_y = pred_y
            args.best_tr_r2 = tr_r2
            args.best_tr_mae = tr_mae
            args.best_tr_mse = tr_mse
            args.best_val_r2 = val_r2
            args.best_val_mae = val_mae
            args.best_val_mse = val_mse
            '''
            tf_writer.add_hparams(
                hparam_dict = vars(args),
                metric_dict={'best_ts_mse' : args.best_mse, 'best_epoch':args.best_epoch}
            )
            '''

        early_stopping(val_loss, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break

    # save result .csv
    for save_iter in range(len(list_val_mse)):
        save_list = {'val_MSE' : save_val_mse[save_iter], 'val_MAE' : save_val_mae[save_iter], 'val_R2' : save_val_r2[save_iter], 'train_MSE' : save_tr_mse[save_iter], 'train_MAE' : save_tr_mae[save_iter], 'train_R2' : save_tr_r2[save_iter], 'test_MSE' : save_mse[save_iter], 'test_MAE' : save_mae[save_iter], 'test_R2' : save_r2[save_iter], 'epoch' : save_epoch[save_iter], 'n_layer' : args.n_layer, 'learning_rate' : args.lr, 'batch_size' : args.batch_size, 'out_dim' : args.out_dim, 'molvec_dim' : args.molvec_dim}
        save_result.append(save_list)
    pd_save = pd.DataFrame(save_result, columns=['train_R2', 'train_MAE', 'train_MSE', 'val_R2', 'val_MAE', 'val_MSE', 'test_R2', 'test_MAE', 'test_MSE', 'n_layer', 'learning_rate', 'batch_size', 'out_dim', 'molvec_dim'])
    pd_save.to_csv(result_file , mode='a', header=False)

    # End of experiments
    te = time.time()
    args.elapsed = te-ts
    args.train_losses = list_train_loss
    args.val_losses = list_val_loss
    args.maes = list_mae
    args.stds = list_std
    return model, args

#Model running

In [ ]:
seed = 950228
np.random.seed(seed)
torch.manual_seed(seed)
start_time = str(time.ctime()).replace(':','-').replace(' ','_')

parser = argparse.ArgumentParser()
args = parser.parse_args("")

# ==== Training Config ==== #
args.batch_size = 128
args.epoch = 10
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ==== Model Architercutre ==== #
args.in_dim = 60
args.in_bond_dim = 10
args.dropout= 0.15
args.out_dim = 256

# ==== Optimizer Config ==== #
args.weight_decay_value = 0.00001
args.lr = 0.001
args.l2_coef = 0.0001
args.radius = 3
args.T = 1
args.per_task_output_units_num = 1 # for regression model
args.output_unit = 1

# writer = Writer(prior_keyword=['n_layer', 'use_bn', 'lr', 'dp_rate', 'emb_train', 'epoch', 'batch_size'])
# writer.clear()

# Define Hyperparameter Search Space
list_lr = [0.0001, 0.0005, 0.001]
list_batch_size = [128]

# partition = {'train': datasets[0], 'val': datasets[1], 'test': datasets[2]}
train_dataloader = DataLoader(gatDataset(datasets[0]), batch_size=args.batch_size, shuffle=True, generator=torch.Generator(device=args.device))
val_dataloader = DataLoader(gatDataset(datasets[1]), batch_size=args.batch_size, shuffle=False, generator=torch.Generator(device=args.device))
test_dataloader = DataLoader(gatDataset(datasets[2]), batch_size=args.batch_size, shuffle=False, generator=torch.Generator(device=args.device))
partition = {'train': train_dataloader, 'val': val_dataloader, 'test': test_dataloader}

for lr in list_lr:
    args.lr = lr

    model, result = experiment(partition, args)
    # writer.write(result)

    print('[Exp {:2}] training tr_r2: {:2.3f}, tr_mae: {:2.3f}, tr_mse: {:2.3f} validation val_r2: {:2.3f}, val_mae: {:2.3f}, val_mse: {:2.3f} test ts_r2: {:2.3f}, ts_mae: {:2.3f}, ts_mse: {:2.3f}, std: {:2.3f} at epoch {:2} took {:3.1f} sec'.format(cnt_exp, result.best_tr_r2, result.best_tr_mae, result.best_tr_mse, result.best_val_r2, result.best_val_mae, result.best_val_mse, result.best_r2, result.best_mae, result.best_mse, result.best_std, result.best_epoch, result.elapsed))